# Veri Seti Bağlama

In [1]:
#Drive bağlantısı yapıldı
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#Kütüphaneler eklendi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras


In [3]:
print(os.listdir("/gdrive/MyDrive/Kidney Cancer"))

DATADIR = '/gdrive/MyDrive/Kidney Cancer'

['Normal', 'Tumor']


In [4]:
CATEGORIES = ["Tumor", "Normal"]
IMG_SIZE = 220

In [5]:
all_data = []
def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [6]:
print("Toplam Görüntü Sayısı: ", len(all_data))


Toplam Görüntü Sayısı:  10000


In [7]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

# %65 Train, %35 Test, %10 Validation

1.   Liste öğesi
2.   Liste öğesi



In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [9]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [10]:
#4.Durum
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(5850, 220, 220, 3)
(3500, 220, 220, 3)
(650, 220, 220, 3)
(5850,)
(3500,)
(650,)


#1.CNN MODEL TASARLAMA ,TRAIN,TEST,VALIDATION DEĞERLERİ,TEST VERİSİNİN ACCURACY,PRECISION,RECALL **VE** F1 SCORE,KARMAŞIKLIK MATRİSİ,CLASSIFICATİON REPORT GÖSTERME

In [11]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [12]:
!pip install keras-tuner
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [13]:
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(hp):
    model = Sequential()

    model.add(Conv2D(16,(3,3),padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16,(3,3),padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(32, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(64, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3) , padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3) , padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.27))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.27))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model


<ipython-input-13-b59240cffd71>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [14]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=3)
]

In [15]:
#Random Search - Hipermparametre Optimizasyonu
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='my_tuner_dir',
    project_name='kidney_cancer_tuning'
)

In [16]:
tuner.search(x_train, train_yCl, validation_data=(x_val, valid_yCl),batch_size=32,verbose=3)

Trial 1 Complete [00h 01m 28s]
val_accuracy: 0.8261538743972778

Best val_accuracy So Far: 0.8261538743972778
Total elapsed time: 00h 01m 28s


In [17]:
#En iyi değeri alma
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

In [18]:
#Model Eğitimi
history = model.fit(x_train,
                    train_yCl,
                    validation_data=(x_val, valid_yCl),
                    batch_size=32,
                    epochs=50,
                    callbacks = callback_list,
                    verbose=3)

Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.80923, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50

Epoch 2: val_accuracy improved from 0.80923 to 0.94615, saving model to model.h5
Epoch 3/50

Epoch 3: val_accuracy improved from 0.94615 to 0.97846, saving model to model.h5
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.97846
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.97846
Epoch 6/50

Epoch 6: val_accuracy did not improve from 0.97846
Epoch 6: early stopping


In [19]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

21/21 [==============================] - 1s 26ms/step - loss: 0.2800 - accuracy: 0.9215
Validation Accuracy:  0.9215384721755981
110/110 [==============================] - 4s 34ms/step - loss: 0.2810 - accuracy: 0.9189
Validation Accuracy:  0.9188571572303772
183/183 [==============================] - 5s 26ms/step - loss: 0.2595 - accuracy: 0.9207
Validation Accuracy:  0.920683741569519


In [20]:
#Test verisinin Accuracy, Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report'u
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test verisi üzerinde tahmin yapma
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# One-hot encoding'i geri çevirme
y_test_classes = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

# Precision
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
print("Test Precision: {:.2f}".format(precision))

# Recall
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
print("Test Recall: {:.2f}".format(recall))

# F1 Score
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
print("Test F1 Score: {:.2f}".format(f1))

# Karmaşıklık Matrisi
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test_classes, y_pred_classes)
print("Classification Report:\n", class_report)



110/110 [==============================] - 3s 26ms/step
Test Accuracy: 91.89%
Test Precision: 0.93
Test Recall: 0.92
Test F1 Score: 0.92
Confusion Matrix:
 [[1502  284]
 [   0 1714]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.84      0.91      1786
           1       0.86      1.00      0.92      1714

    accuracy                           0.92      3500
   macro avg       0.93      0.92      0.92      3500
weighted avg       0.93      0.92      0.92      3500

